# Humble Chuck's SQL Helper

### Easily create and edit databases and tables using pandas

First, we'll import the required libraries for this demo.

- I have mysql_helper stored in my python path, but you can access a copy of the file in this repo.
- The 'config' file contains my host, user, and password for MY_SQL

In [1]:
from Humble_Chuck import mysql_helper1 as sql
import config
import pandas as pd

- 'game.csv' and 'weather.csv' contain data from 50 random MLB games from the 2019 season
- data was pulled from the MLB Stats API

In [2]:
games = pd.read_csv("games.csv")
weather = pd.read_csv("weather.csv")

In [3]:
display(games.head())
display(weather.head())

,pk,type,doubleHeader,id,gamedayType,tiebreaker,gameNumber,calendarEventID,season,seasonDisplay,home_id,away_id,venue_id
0,565514,R,N,2019/04/10/clemlb-detmlb-1,P,N,1,14-565514-2019-04-10,2019,2019,116,114,2394
1,567014,R,N,2019/08/02/kcamlb-minmlb-1,P,N,1,14-567014-2019-08-02,2019,2019,142,118,3312
2,565522,R,N,2019/05/03/kcamlb-detmlb-1,P,N,1,14-565522-2019-05-03,2019,2019,116,118,2394
3,565576,R,N,2019/09/01/minmlb-detmlb-1,P,N,1,14-565576-2019-09-01,2019,2019,116,142,2394
4,565628,R,N,2019/05/22/chamlb-houmlb-1,P,N,1,14-565628-2019-05-22,2019,2019,117,145,2392


,_condition,temp,wind,pk
0,Cloudy,42,"10 mph, L To R",565514
1,Partly Cloudy,83,"7 mph, Out To LF",567014
2,Cloudy,51,"10 mph, Out To RF",565522
3,Cloudy,63,"5 mph, L To R",565576
4,Roof Closed,73,"0 mph, None",565628


## Now for the main event:

#### mysql_helper allows us to easily move from DataFrames to SQL tables 

The first step is to instantiate a MY_SQL connection using the parameters found in the config file.

In [4]:
flatiron = sql.Connection(config.host,config.user,config.password)

- the query method for a Connection allows us to query our database with typical SQL syntax
- the add_DB and drop_DB methods are included for convenience
- you can also call the cnx or cursor methods to use a MY_SQL connection or cursor as normal 

In [5]:
flatiron.query('show databases')

[('MLB_Stats',),
 ('Yelp_data',),
 ('employees',),
 ('information_schema',),
 ('innodb',),
 ('mysql',),
 ('performance_schema',),
 ('practice',),
 ('production',),
 ('sales',),
 ('students',),
 ('sys',)]

In [6]:
flatiron.add_DB('example')

CREATE DATABASE IF NOT EXISTS example


In [7]:
# now we can instantiate a direct connection to a database using the DataBase sub-class
example = sql.DataBase(flatiron,'example')
example.query('show tables') #we don't have any tables though

[]

#### Now that we have our example database, we can go forth to add a table using nothing more than our established DataFrames 

- The 'primary_key' optional argument allows us to specify a primary key

In [8]:
example.table_fromDf(games,'games','pk')

CREATE TABLE IF NOT EXISTS games (pk INT (64), type VARCHAR (250), doubleHeader VARCHAR (250), id VARCHAR (250), gamedayType VARCHAR (250), tiebreaker VARCHAR (250), gameNumber INT (64), calendarEventID VARCHAR (250), season INT (64), seasonDisplay INT (64), home_id INT (64), away_id INT (64), venue_id INT (64), PRIMARY KEY (pk));
Creating a new table
OK


In [9]:
example.table_fromDf(weather,'weather')

CREATE TABLE IF NOT EXISTS weather (_condition VARCHAR (250), temp INT (64), wind VARCHAR (250), pk INT (64));
Creating a new table
OK


- The insert_fromDF method will insert data from a pandas DF into a specified SQL table 

- The insert_fromDF_iteration will method do the same, but for each individual record. 
    - That way, you can see your progress with print statements. 

In [10]:
example.insert_fromDf_iteration(games,'games')
example.insert_fromDf_iteration(weather, 'weather')

(565514, 'R', 'N', '2019/04/10/clemlb-detmlb-1', 'P') succesfully inserted
(567014, 'R', 'N', '2019/08/02/kcamlb-minmlb-1', 'P') succesfully inserted
(565522, 'R', 'N', '2019/05/03/kcamlb-detmlb-1', 'P') succesfully inserted
(565576, 'R', 'N', '2019/09/01/minmlb-detmlb-1', 'P') succesfully inserted
(565628, 'R', 'N', '2019/05/22/chamlb-houmlb-1', 'P') succesfully inserted
(567516, 'R', 'S', '2019/08/03/bosmlb-nyamlb-1', 'P') succesfully inserted
(565618, 'R', 'N', '2019/04/28/clemlb-houmlb-1', 'P') succesfully inserted
(567505, 'R', 'N', '2019/07/14/tormlb-nyamlb-1', 'P') succesfully inserted
(566277, 'R', 'N', '2019/03/30/sfnmlb-sdnmlb-1', 'P') succesfully inserted
(566256, 'R', 'N', '2019/09/25/chnmlb-pitmlb-1', 'P') succesfully inserted
(567139, 'R', 'N', '2019/09/29/miamlb-phimlb-1', 'P') succesfully inserted
(566976, 'R', 'N', '2019/04/30/houmlb-minmlb-1', 'P') succesfully inserted
(565443, 'R', 'N', '2019/06/01/tormlb-colmlb-1', 'P') succesfully inserted
(566842, 'R', 'N', '2019/

Now we can make a new DataFrame directly from our database 

In [11]:
example.query_to_df("""select
                    g.pk,
                    g.season,
                    g.home_id,
                    g.away_id,
                    g.venue_id,
                    w._condition,
                    w.temp,
                    w.wind
                    from games g inner join weather w on g.pk=w.pk; """)

,pk,season,home_id,away_id,venue_id,_condition,temp,wind
0,565514,2019,116,114,2394,Cloudy,42,"10 mph, L To R"
1,567014,2019,142,118,3312,Partly Cloudy,83,"7 mph, Out To LF"
2,565522,2019,116,118,2394,Cloudy,51,"10 mph, Out To RF"
3,565576,2019,116,142,2394,Cloudy,63,"5 mph, L To R"
4,565628,2019,117,145,2392,Roof Closed,73,"0 mph, None"
5,567516,2019,147,111,3313,Partly Cloudy,84,"7 mph, R To L"
6,565618,2019,117,114,2392,Partly Cloudy,80,"9 mph, Out To LF"
7,567505,2019,147,141,3313,Sunny,88,"15 mph, L To R"
8,566277,2019,135,137,2680,Clear,70,"13 mph, L To R"
9,566256,2019,134,112,31,Partly Cloudy,77,"9 mph, Out To CF"
